In [1]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Image": photo,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", "")
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    df = pd.DataFrame(all_data)
    df.drop(columns=["Image"], inplace=True)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    display_images()

def prompt_select_files(event=None):
    read_dicom_files()

def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

def animate_label_color(label, colors, index=0):
    label.configure(text_color=colors[index])
    next_index = (index + 1) % len(colors)
    label.after(500, animate_label_color, label, colors, next_index)

def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

    if not all_data:
        label = ctk.CTkLabel(
            content_frame,
            text="\n Click Here To Select DICOM Files",
            font=ctk.CTkFont(size=28, weight="bold"),
            justify="center",
            anchor="center",
            cursor="hand2"
        )
        label.pack(expand=True, fill="both", padx=50, pady=50)
        label.bind("<Button-1>", lambda e: read_dicom_files())

        # ألوان مريحة تتغير تدريجياً
        colors = ["#3a86ff", "#4dabf5", "#71b8ff", "#4dabf5"]
        animate_label_color(label, colors)

        return


    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])
    dose_summary = defaultdict(lambda: defaultdict(float))
    date_ranges = defaultdict(lambda: defaultdict(list))

    columns = 4
    for index, data in enumerate(sorted_data):
        row = index // columns
        col = index % columns

        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)
        else:
            ctk.CTkLabel(frame, text="No Image").pack()

        info = (
            f"👤 Name: {data['Name']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"☢ Dose Result (mSv): {data['mSv']:.2f}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)

        start_date = data["Date"]
        end_date = start_date + timedelta(days=365)
        patient_name = data["Name"]
        for d in all_data:
            if d["Name"] == patient_name and start_date <= d["Date"] <= end_date:
                year = start_date.year
                dose_summary[patient_name][year] += d["mSv"]
                date_ranges[patient_name][year].append(d["Date"])

    summary_box.delete("1.0", "end")
    summary_box.insert("end", "\n📊 Sum of Dose per Year:\n")
    for name in dose_summary:
        for year, total_dose in dose_summary[name].items():
            dates = sorted(date_ranges[name][year])
            start = dates[0].strftime("%Y-%m-%d") if dates else "N/A"
            end = dates[-1].strftime("%Y-%m-%d") if dates else "N/A"
            summary_box.insert("end", f"{name} - {year}: {total_dose:.2f} mSv (From {start} to {end})\n")

def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_textbox.delete("1.0", "end")
            hl7_textbox.insert("end", hl7_content)
            return

    messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
    hl7_textbox.delete("1.0", "end")

def clear_images():
    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()
    summary_box.delete("1.0", "end")
    hl7_textbox.delete("1.0", "end")
    display_images()  # اعادة عرض رسالة رفع الملفات

root = ctk.CTk()
root.title("DICOM Viewer + HL7 + Summary")
root.geometry("1250x850")
root.resizable(True, True)

try:
    bg_image = Image.open("g.jpg").resize((1250, 850))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = ctk.CTkLabel(root, image=bg_photo, text="")
    bg_label.place(relx=0, rely=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not loaded: {e}")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.05)

clear_button = ctk.CTkButton(root, text="🗑 Clear All", command=clear_images)
clear_button.place(relx=0.15, rely=0.05)

ctk.CTkLabel(root, text="🔍 Search HL7 by Name:").place(relx=0.35, rely=0.05)
search_entry = ctk.CTkEntry(root, width=180)
search_entry.place(relx=0.52, rely=0.05)
search_button = ctk.CTkButton(root, text="Search", command=search_hl7_message)
search_button.place(relx=0.68, rely=0.05)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.8, rely=0.05)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_images())
sort_optionmenu.place(relx=0.85, rely=0.05)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.12, relwidth=0.96, relheight=0.55)

hl7_textbox = ctk.CTkTextbox(root)
hl7_textbox.place(relx=0.02, rely=0.7, relwidth=0.96, relheight=0.1)

summary_box = ctk.CTkTextbox(root)
summary_box.place(relx=0.02, rely=0.82, relwidth=0.96, relheight=0.1)

display_images()  # عند بداية التشغيل تظهر رسالة رفع الملفات

root.mainloop()


C:\Users\maryam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


In [ ]:
# DICOM HL7 Viewer Advanced

import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

CSV_FILE = "patients_data.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# ========== HL7 Message Generator ==========
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", ""),
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "PatientID", "")
    modality = getattr(ds, "Modality", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{study_id}||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||{modality}
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

# ========== File Handling ==========
def read_single_dicom():
    file = filedialog.askopenfilename(filetypes=[("DICOM files", "*.dcm")])
    if not file:
        return

    try:
        ds = pydicom.dcmread(file)
        name = str(getattr(ds, "PatientName", "Unknown"))
        date_str = getattr(ds, "StudyDate", "00000000")
        date_obj = datetime.strptime(date_str, "%Y%m%d") if date_str != "" else datetime.now()
        ctdi = float(getattr(ds, "CTDIvol", 0))
        modality = getattr(ds, "Modality", "Unknown")
        msv = ctdi * 0.014
        patient_id = getattr(ds, "PatientID", "")

        row = {
            "ID": patient_id,
            "Name": name,
            "Date": date_obj,
            "Modality": modality,
            "CTDIvol": ctdi,
            "mSv": msv
        }

        df = pd.DataFrame([row])
        if os.path.exists(CSV_FILE):
            df.to_csv(CSV_FILE, mode='a', index=False, header=False)
        else:
            df.to_csv(CSV_FILE, index=False)

        hl7_msg = convert_to_hl7(ds, msv)
        hl7_filename = f"{HL7_DIR}/{patient_id}_{date_obj.strftime('%Y%m%d')}.hl7"
        with open(hl7_filename, "w") as f:
            f.write(hl7_msg)

        messagebox.showinfo("Success", "DICOM file imported and data saved.")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to process file.\n{e}")

# ========== Viewer ==========
def read_multiple_dicom():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return
    all_data.clear()
    for f in files:
        try:
            ds = pydicom.dcmread(f)
            name = str(getattr(ds, "PatientName", "Unknown"))
            date = getattr(ds, "StudyDate", "00000000")
            date_obj = datetime.strptime(date, "%Y%m%d")
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            modality = getattr(ds, "Modality", "Unknown")
            msv = ctdi * 0.014
            img = Image.fromarray(ds.pixel_array)
            img.thumbnail((200, 200))
            img_tk = ImageTk.PhotoImage(img)

            all_data.append({
                "ID": getattr(ds, "PatientID", ""),
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "mSv": msv,
                "Modality": modality,
                "Image": img_tk
            })
        except Exception as e:
            print(f"Error: {e}")

    display_cases()

def display_cases():
    for widget in cases_frame.winfo_children():
        widget.destroy()

    sorted_data = sorted(all_data, key=lambda x: x[sort_var.get()])

    for i, d in enumerate(sorted_data[:4]):
        frame = ctk.CTkFrame(cases_frame)
        frame.grid(row=i//2, column=i%2, padx=10, pady=10)
        ctk.CTkLabel(frame, text=f"ID: {d['ID']}\nName: {d['Name']}\nDate: {d['Date'].date()}\nModality: {d['Modality']}\nCTDIvol: {d['CTDIvol']} mGy\nmSv: {d['mSv']:.2f}").pack()
        ctk.CTkLabel(frame, image=d["Image"], text="").pack()

# ========== HL7 Search ==========
def search_hl7():
    pwd = simpledialog.askstring("Password", "Enter password:", show='*')
    if pwd != "admin123":
        messagebox.showerror("Access Denied", "Incorrect password")
        return

    pid = simpledialog.askstring("Search HL7", "Enter Patient ID:")
    for fname in os.listdir(HL7_DIR):
        if pid in fname:
            with open(os.path.join(HL7_DIR, fname)) as f:
                hl7_view.delete("1.0", "end")
                hl7_view.insert("end", f.read())
                return
    messagebox.showinfo("Not Found", "No HL7 message found for this ID.")

# ========== GUI Layout ==========
root = ctk.CTk()
root.geometry("1200x850")
root.title("Advanced DICOM Viewer + HL7")

ctk.CTkButton(root, text="📥 Import Single DICOM", command=read_single_dicom).place(relx=0.03, rely=0.02)
ctk.CTkButton(root, text="📂 Select Cases", command=read_multiple_dicom).place(relx=0.25, rely=0.02)
ctk.CTkButton(root, text="🔐 HL7 Messages", command=search_hl7).place(relx=0.45, rely=0.02)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort By:").place(relx=0.65, rely=0.02)
ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_cases()).place(relx=0.72, rely=0.02)

cases_frame = ctk.CTkScrollableFrame(root)
cases_frame.place(relx=0.03, rely=0.08, relwidth=0.94, relheight=0.6)

hl7_view = ctk.CTkTextbox(root)
hl7_view.place(relx=0.03, rely=0.7, relwidth=0.94, relheight=0.25)

root.mainloop()


In [ ]:
# هذا الكود هو تكملة وتطوير للمشروع السابق مع دعم اختيار حالتين أو أربع حالات لعرضهم
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

all_data = []
selected_cases = []

# ---------- الوظائف ----------
def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            study_type = str(getattr(ds, "Modality", "Unknown"))
            date_str = getattr(ds, "StudyDate", "00000000")
            study_id = str(getattr(ds, "PatientID", "Unknown"))
            gender = getattr(ds, "PatientSex", "")
            dob = getattr(ds, "PatientBirthDate", "")

            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((150, 150))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            data = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "mSv": msv,
                "StudyType": study_type,
                "Image": photo,
                "Sex": gender,
                "DOB": dob,
                "ID": study_id,
                "Path": path
            }

            all_data.append(data)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_cases()

def display_cases():
    for widget in content_frame.winfo_children():
        widget.destroy()

    for i, data in enumerate(all_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=i//3, column=i%3, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 {data['Name']}\n"
            f"🆔 ID: {data['ID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧪 Type: {data['StudyType']}\n"
            f"☢ CTDIvol: {data['CTDIvol']} mGy\n"
            f"☢ mSv: {data['mSv']:.2f} mSv"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack()

        chk = ctk.CTkCheckBox(frame, text="اختيار الحالة",
                              command=lambda d=data: toggle_selection(d))
        chk.pack(pady=5)

def toggle_selection(data):
    if data in selected_cases:
        selected_cases.remove(data)
    elif len(selected_cases) < 4:
        selected_cases.append(data)
    else:
        messagebox.showwarning("Max Reached", "يمكنك فقط اختيار حالتين إلى أربع حالات.")

def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "يجب اختيار حالتين أو أربع حالات فقط.")
        return

    win = ctk.CTkToplevel(root)
    win.title("📋 الحالات المختارة")
    win.geometry("900x600")

    for i, data in enumerate(selected_cases):
        frame = ctk.CTkFrame(win)
        frame.grid(row=i//2, column=i%2, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        details = (
            f"👤 الاسم: {data['Name']}\n"
            f"🆔 ID: {data['ID']}\n"
            f"📅 التاريخ: {data['Date'].date()}\n"
            f"🎯 النوع: {data['StudyType']}\n"
            f"☢ CTDIvol: {data['CTDIvol']} mGy\n"
            f"☢ الجرعة (mSv): {data['mSv']:.2f} mSv\n"
            f"👤 النوع: {data['Sex']}\n"
            f"🎂 تاريخ الميلاد: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=details, justify="left").pack(pady=5)

# ---------- واجهة البرنامج ----------
root = ctk.CTk()
root.title("📁 DICOM Viewer & Selector")
root.geometry("1280x800")

select_button = ctk.CTkButton(root, text="📂 اختر ملفات DICOM", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.86)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# تحويل بيانات DICOM إلى رسالة HL7

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


# قراءة ملفات DICOM وتخزين البيانات فقط

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_text_data()


# عرض البيانات فقط بدون صور

def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])
    
    for index, data in enumerate(sorted_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=index, column=0, pady=5, padx=10, sticky="ew")

        info = (
            f"👤 Name: {data['Name']} | 🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()} | 🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv | CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm | Sex: {data['Sex']} | DOB: {data['DOB']}"
        )

        label = ctk.CTkLabel(frame, text=info, anchor="w", justify="left")
        label.pack(side="left", padx=10)

        select_btn = ctk.CTkButton(frame, text="✔ Select", width=80,
                                   command=lambda d=data: select_case(d))
        select_btn.pack(side="right", padx=10)


def select_case(data):
    if data in selected_cases:
        messagebox.showinfo("Already Selected", "This case is already selected.")
        return
    if len(selected_cases) >= 4:
        messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
        return
    selected_cases.append(data)
    messagebox.showinfo("Selected", "Case selected successfully.")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# تحويل بيانات DICOM إلى رسالة HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    for index, data in enumerate(sorted_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=index, column=0, pady=5, padx=10, sticky="ew")

        info = (
            f"👤 Name: {data['Name']} | 🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()} | 🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv | CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm | Sex: {data['Sex']} | DOB: {data['DOB']}"
        )

        label = ctk.CTkLabel(frame, text=info, anchor="w", justify="left")
        label.pack(side="left", padx=10)

        select_btn = ctk.CTkButton(frame, text="✔ Select", width=80,
                                   command=lambda d=data: select_case(d))
        select_btn.pack(side="right", padx=5)

        delete_btn = ctk.CTkButton(frame, text="🗑 Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.pack(side="right", padx=5)

        hl7_btn = ctk.CTkButton(frame, text="📨 HL7 Message", width=110,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.pack(side="right", padx=5)


def select_case(data):
    if data in selected_cases:
        messagebox.showinfo("Already Selected", "This case is already selected.")
        return
    if len(selected_cases) >= 4:
        messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
        return
    selected_cases.append(data)
    messagebox.showinfo("Selected", "Case selected successfully.")


def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    if data in selected_cases:
        selected_cases.remove(data)
    display_text_data()


def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []

# تحويل بيانات DICOM إلى رسالة HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""


def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_text_data()


def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    for index, data in enumerate(sorted_data):
        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=index, column=0, pady=5, padx=10, sticky="ew")

        info = (
            f"👤 Name: {data['Name']} | 🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()} | 🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv | CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm | Sex: {data['Sex']} | DOB: {data['DOB']}"
        )

        label = ctk.CTkLabel(frame, text=info, anchor="w", justify="left")
        label.pack(side="left", padx=10)

        select_btn = ctk.CTkButton(frame, text="✔ Select", width=80,
                                   command=lambda d=data: select_case(d))
        select_btn.pack(side="right", padx=5)

        delete_btn = ctk.CTkButton(frame, text="🗑 Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.pack(side="right", padx=5)

        hl7_btn = ctk.CTkButton(frame, text="📨 HL7 Message", width=110,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.pack(side="right", padx=5)


def select_case(data):
    if data in selected_cases:
        messagebox.showinfo("Already Selected", "This case is already selected.")
        return
    if len(selected_cases) >= 4:
        messagebox.showwarning("Limit Reached", "You can only select up to 4 cases.")
        return
    selected_cases.append(data)
    messagebox.showinfo("Selected", "Case selected successfully.")


def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    if data in selected_cases:
        selected_cases.remove(data)
    display_text_data()


def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")


def show_selected_cases():
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)


root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1200x850")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁‍🗨 عرض الحالات المختارة", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_text_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog, IntVar
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_vars = []  # to hold IntVars for checkboxes

# Convert DICOM to HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_vars.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_table_data()

def display_table_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # headers
    headers = ["Select", "Name", "ID", "Date", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)", "Sex", "DOB", "HL7", "Delete"]

    # Sort data
    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    # Create header labels
    for col, header in enumerate(headers):
        label = ctk.CTkLabel(content_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        label.grid(row=0, column=col, padx=5, pady=5, sticky="nsew")

    # Clear selected_vars and recreate
    selected_vars.clear()

    for row, data in enumerate(sorted_data, start=1):
        var = IntVar()
        selected_vars.append((var, data))
        checkbox = ctk.CTkCheckBox(content_frame, variable=var)
        checkbox.grid(row=row, column=0, padx=5, pady=5, sticky="nsew")

        ctk.CTkLabel(content_frame, text=data["Name"]).grid(row=row, column=1, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["StudyID"]).grid(row=row, column=2, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=3, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=4, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']}").grid(row=row, column=5, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['DLP']}").grid(row=row, column=6, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Sex"]).grid(row=row, column=7, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["DOB"]).grid(row=row, column=8, padx=5, pady=5, sticky="nsew")

        hl7_btn = ctk.CTkButton(content_frame, text="View HL7", width=80,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.grid(row=row, column=9, padx=5, pady=5)

        delete_btn = ctk.CTkButton(content_frame, text="Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.grid(row=row, column=10, padx=5, pady=5)

def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    # Also remove from selected_vars any var related to this data
    for var, d in selected_vars[:]:
        if d == data:
            selected_vars.remove((var, d))
    display_table_data()

def show_selected_cases():
    selected = [d for var, d in selected_vars if var.get() == 1]
    if len(selected) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.attributes('-topmost', True)  # Keep window on top

    for idx, data in enumerate(selected):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, corner_radius=10)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].strftime('%Y-%m-%d')}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left", font=ctk.CTkFont(size=14)).pack(pady=5, padx=10)

root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1300x900")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁 View Selected Cases", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_table_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()


In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog, IntVar
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_vars = []  # to hold IntVars for checkboxes

# Convert DICOM to HL7
def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_vars.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            img = None
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                img = ImageTk.PhotoImage(img)

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", ""),
                "Image": img,
                "Path": path
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    display_table_data()
# زر جديد لعرض HL7 لجميع الحالات المختارة (2 أو 4)
def show_selected_hl7_messages():
    selected = [d for var, d in selected_vars if var.get() == 1]
    if len(selected) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 messages:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    hl7_texts = []
    for data in selected:
        filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
        if os.path.exists(filename):
            with open(filename, "r") as f:
                hl7 = f.read()
            hl7_texts.append(f"--- HL7 for {data['Name']} ({data['Date'].strftime('%Y-%m-%d')}) ---\n{hl7}\n")
        else:
            hl7_texts.append(f"HL7 message not found for {data['Name']}.\n")

    hl7_window = ctk.CTkToplevel()
    hl7_window.title("HL7 Messages Viewer")
    hl7_window.geometry("800x600")
    hl7_window.attributes('-topmost', True)

    text_widget = ctk.CTkTextbox(hl7_window, width=780, height=580)
    text_widget.pack(padx=10, pady=10, fill="both", expand=True)
    text_widget.insert("0.0", "\n\n".join(hl7_texts))
    text_widget.configure(state="disabled")

# ... 

# فوق الجدول، نضيف زر HL7 مع الأزرار الأخرى
hl7_button = ctk.CTkButton(root, text="🗒 View HL7 for Selected", command=show_selected_hl7_messages)
hl7_button.place(relx=0.42, rely=0.03)
def display_table_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # headers
    headers = ["Select", "Name", "ID", "Date", "Dose (mSv)", "CTDIvol (mGy)", "DLP (mGy·cm)", "Sex", "DOB", "HL7", "Delete"]

    # Sort data
    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    # Create header labels
    for col, header in enumerate(headers):
        label = ctk.CTkLabel(content_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        label.grid(row=0, column=col, padx=5, pady=5, sticky="nsew")

    # Clear selected_vars and recreate
    selected_vars.clear()

    for row, data in enumerate(sorted_data, start=1):
        var = IntVar()
        selected_vars.append((var, data))
        checkbox = ctk.CTkCheckBox(content_frame, variable=var)
        checkbox.grid(row=row, column=0, padx=5, pady=5, sticky="nsew")

        ctk.CTkLabel(content_frame, text=data["Name"]).grid(row=row, column=1, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["StudyID"]).grid(row=row, column=2, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Date"].strftime("%Y-%m-%d")).grid(row=row, column=3, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=4, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['CTDIvol']}").grid(row=row, column=5, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=f"{data['DLP']}").grid(row=row, column=6, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["Sex"]).grid(row=row, column=7, padx=5, pady=5, sticky="nsew")
        ctk.CTkLabel(content_frame, text=data["DOB"]).grid(row=row, column=8, padx=5, pady=5, sticky="nsew")

        hl7_btn = ctk.CTkButton(content_frame, text="View HL7", width=80,
                                command=lambda d=data: show_hl7_message(d))
        hl7_btn.grid(row=row, column=9, padx=5, pady=5)

        delete_btn = ctk.CTkButton(content_frame, text="Delete", width=80,
                                   command=lambda d=data: delete_case(d))
        delete_btn.grid(row=row, column=10, padx=5, pady=5)

def show_hl7_message(data):
    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}.hl7"
    if os.path.exists(filename):
        with open(filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def delete_case(data):
    if data in all_data:
        all_data.remove(data)
    # Also remove from selected_vars any var related to this data
    for var, d in selected_vars[:]:
        if d == data:
            selected_vars.remove((var, d))
    display_table_data()

def show_selected_cases():
    selected = [d for var, d in selected_vars if var.get() == 1]
    if len(selected) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select exactly 2 or 4 cases.")
        return

    top = ctk.CTkToplevel()
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.attributes('-topmost', True)  # Keep window on top

    for idx, data in enumerate(selected):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, corner_radius=10)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].strftime('%Y-%m-%d')}\n"
            f"🧬 Type: CT\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left", font=ctk.CTkFont(size=14)).pack(pady=5, padx=10)

root = ctk.CTk()
root.title("DICOM Viewer - Select & Display")
root.geometry("1300x900")

select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.03)

view_button = ctk.CTkButton(root, text="👁 View Selected Cases", command=show_selected_cases)
view_button.place(relx=0.22, rely=0.03)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.7, rely=0.03)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_table_data())
sort_optionmenu.place(relx=0.78, rely=0.03)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.85)

root.mainloop()
